In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets

import pennylane as qml
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import sklearn.decomposition

import keras.metrics
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('DrDoS_SSDP_features_removed.csv', skiprows=[i for i in range(1,141550)], skipfooter=141547, engine="python")

n_features = 2
x = StandardScaler().fit_transform(np.array(data.drop(columns=['Label'])))
y = np.array(data['Label'].astype('category').cat.codes.astype(int))

np.random.seed(1967)
x, y = zip(*np.random.permutation(list(zip(x, y))))

pca = sklearn.decomposition.PCA(n_components=n_features)
pca.fit(x)
x = pca.transform(x)

minmax_scale = MinMaxScaler((-1, 1)).fit(x)
x = minmax_scale.transform(x)


trainX, testX, trainy, testy = train_test_split(x, y, stratify=y, test_size=0.3, random_state=109)

trainy = tf.one_hot(trainy, depth=2)
testy = tf.one_hot(testy, depth=2)

In [5]:
n_qubits = 2
layers = 1
data_dimension = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


weight_shapes = {"weights": (layers,n_qubits,3)}

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(n_qubits,activation='relu',input_dim=2))
model.add(qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits))
model.add(tf.keras.layers.Dense(data_dimension, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=["accuracy"])

In [6]:
%load_ext memory_profiler

In [7]:
%%time

history = model.fit(trainX, trainy, validation_data=(trainX, trainy), epochs=30, batch_size=5)

Epoch 1/30
413/413 [==============================] - 146s 352ms/step - loss: 0.4131 - accuracy: 0.8349 - val_loss: 0.0577 - val_accuracy: 0.9995
Epoch 2/30
413/413 [==============================] - 156s 377ms/step - loss: 0.0238 - accuracy: 0.9995 - val_loss: 0.0121 - val_accuracy: 0.9995
Epoch 3/30
413/413 [==============================] - 144s 348ms/step - loss: 0.0097 - accuracy: 0.9995 - val_loss: 0.0078 - val_accuracy: 0.9995
Epoch 4/30
413/413 [==============================] - 145s 351ms/step - loss: 0.0073 - accuracy: 0.9995 - val_loss: 0.0064 - val_accuracy: 0.9995
Epoch 5/30
413/413 [==============================] - 143s 347ms/step - loss: 0.0063 - accuracy: 0.9995 - val_loss: 0.0057 - val_accuracy: 0.9995
Epoch 6/30
413/413 [==============================] - 143s 347ms/step - loss: 0.0057 - accuracy: 0.9995 - val_loss: 0.0051 - val_accuracy: 0.9995
Epoch 7/30
413/413 [==============================] - 127s 309ms/step - loss: 0.0053 - accuracy: 0.9995 - val_loss: 0.0048 -

In [10]:
predy = model.predict(testX)
predy

array([[2.4881068e-05, 9.9997509e-01],
       [3.2592427e-05, 9.9996746e-01],
       [9.9999022e-01, 9.7645880e-06],
       ...,
       [6.0318478e-05, 9.9993968e-01],
       [1.1297456e-04, 9.9988699e-01],
       [2.3335928e-05, 9.9997663e-01]], dtype=float32)

In [12]:
rounded_labels_real

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,

In [13]:
rounded_labels_pred

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,

In [18]:
from sklearn.metrics import classification_report, recall_score
from sklearn.metrics import f1_score, accuracy_score, precision_score, make_scorer

#Metrics
classification = classification_report(rounded_labels_real, rounded_labels_pred)
confusion = confusion_matrix(rounded_labels_real, rounded_labels_pred)
    
    # Accuracy
accuracy = round(accuracy_score(rounded_labels_real, rounded_labels_pred)*100,5)
    
    # Recall
recall = round(recall_score(rounded_labels_real, rounded_labels_pred, average='macro')*100,5)
    
    # Precision
precision = round(precision_score(rounded_labels_real, rounded_labels_pred, average='weighted')*100,5)
    
    # F1
f1 = round(f1_score(rounded_labels_real, rounded_labels_pred, average='weighted')*100,5)
    

In [19]:
print(accuracy)
print(recall)
print(precision)
print(f1)
print(1-(accuracy/100))

99.88701
99.88713
99.88726
99.88701
0.001129900000000017
CPU times: user 43min 50s, sys: 11.8 s, total: 44min 2s 
 Wall time: 43min 45s


In [20]:
print(qnode.draw())

 0: ──RX(0.067)──Rot(-0.071, -0.033, 0.591)──╭C──╭X──┤ ⟨Z⟩ 
 1: ──RX(1.317)──Rot(1.571, -1.375, 0.309)───╰X──╰C──┤ ⟨Z⟩ 

